In [1]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/Age_Datasets/Boomer_stroke.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,1,Male,67,0,1,Yes,Private,Urban,228.69,36.6,Former,1
1,2,Female,61,0,0,Yes,Self-employed,Rural,202.21,NaN,Never,1
2,3,Male,80,0,1,Yes,Private,Rural,105.92,32.5,Never,1
3,5,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,Never,1
4,7,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1
5,8,Female,69,0,0,No,Private,Urban,94.39,22.8,Never,1
6,10,Female,78,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
7,12,Female,61,0,1,Yes,Govt_job,Rural,120.46,36.8,Current,1
8,14,Male,78,0,1,Yes,Private,Urban,219.84,NaN,Unknown,1
9,15,Female,79,0,1,Yes,Private,Urban,214.09,28.2,Never,1


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,Former,1
1,Female,61,0,0,Yes,Self-employed,Rural,202.21,NaN,Never,1
2,Male,80,0,1,Yes,Private,Rural,105.92,32.5,Never,1
3,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,Never,1
4,Male,74,1,1,Yes,Private,Rural,70.09,27.4,Never,1


In [28]:
medical_df['Stroke'].value_counts()

0    1034
1     154
Name: Stroke, dtype: int64

In [4]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [5]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(1188, 13)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
9,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [6]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188 entries, 0 to 1187
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            1188 non-null   float64
 1   Gender_Male              1188 non-null   float64
 2   Ever_Married_No          1188 non-null   float64
 3   Ever_Married_Yes         1188 non-null   float64
 4   Work_Type_Govt_job       1188 non-null   float64
 5   Work_Type_Private        1188 non-null   float64
 6   Work_Type_Self-employed  1188 non-null   float64
 7   Residence_Type_Rural     1188 non-null   float64
 8   Residence_Type_Urban     1188 non-null   float64
 9   Smoker_Current           1188 non-null   float64
 10  Smoker_Former            1188 non-null   float64
 11  Smoker_Never             1188 non-null   float64
 12  Smoker_Unknown           1188 non-null   float64
dtypes: float64(13)
memory usage: 120.8 KB


In [7]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188 entries, 0 to 1187
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              1188 non-null   float64
 1   Ever_Married_Yes         1188 non-null   float64
 2   Work_Type_Govt_job       1188 non-null   float64
 3   Work_Type_Private        1188 non-null   float64
 4   Work_Type_Self-employed  1188 non-null   float64
 5   Residence_Type_Urban     1188 non-null   float64
 6   Smoker_Current           1188 non-null   float64
 7   Smoker_Former            1188 non-null   float64
 8   Smoker_Never             1188 non-null   float64
 9   Smoker_Unknown           1188 non-null   float64
dtypes: float64(10)
memory usage: 92.9 KB


In [8]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(1188, 16)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,67,0,1,228.69,36.6,1,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,61,0,0,202.21,NaN,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80,0,1,105.92,32.5,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,79,1,0,174.12,24.0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,74,1,1,70.09,27.4,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,69,0,0,94.39,22.8,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
6,78,0,0,58.57,24.2,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7,61,0,1,120.46,36.8,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,78,0,1,219.84,NaN,1,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
9,79,0,1,214.09,28.2,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [9]:
# Create SimpleImputer instance to replace missing BMI feature values with median BMI
imputer = SimpleImputer(strategy='mean')

# Fit SimpleImputer & transform data
med_transformed = imputer.fit_transform(medical_df)

In [10]:
# Add SimpleImputer outcome to dataframe
med_df_transformed = pd.DataFrame(med_transformed, columns=medical_df.columns)
print(med_df_transformed.shape)
med_df_transformed.head(10)

(1188, 16)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Private,Work_Type_Self-employed,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,67.0,0.0,1.0,228.69,36.600000,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,61.0,0.0,0.0,202.21,30.070054,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80.0,0.0,1.0,105.92,32.500000,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,79.0,1.0,0.0,174.12,24.000000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,74.0,1.0,1.0,70.09,27.400000,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,69.0,0.0,0.0,94.39,22.800000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
6,78.0,0.0,0.0,58.57,24.200000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7,61.0,0.0,1.0,120.46,36.800000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,78.0,0.0,1.0,219.84,30.070054,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
9,79.0,0.0,1.0,214.09,28.200000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [11]:
med_df_transformed.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Private          0
Work_Type_Self-employed    0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [12]:
# Create feature & target datasets
X = med_df_transformed.drop(columns=['Stroke'])
y = med_df_transformed['Stroke']
print(X.shape)
print(y.shape)

(1188, 15)
(1188,)


In [13]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current progress: 796
Current pr

In [14]:
recalls

[0.6129032258064516,
 0.6129032258064516,
 0.6129032258064516,
 0.7096774193548387,
 0.6774193548387096,
 0.7096774193548387,
 0.6774193548387096,
 0.41935483870967744,
 0.41935483870967744,
 0.5806451612903226,
 0.5806451612903226,
 0.6129032258064516,
 0.5806451612903226,
 0.5483870967741935,
 0.7096774193548387,
 0.5161290322580645,
 0.7419354838709677,
 0.5483870967741935,
 0.6129032258064516,
 0.7096774193548387,
 0.6774193548387096,
 0.7096774193548387,
 0.6129032258064516,
 0.6129032258064516,
 0.5161290322580645,
 0.4838709677419355,
 0.6129032258064516,
 0.7741935483870968,
 0.6774193548387096,
 0.5161290322580645,
 0.6129032258064516,
 0.4838709677419355,
 0.5483870967741935,
 0.7741935483870968,
 0.7096774193548387,
 0.6451612903225806,
 0.6451612903225806,
 0.6451612903225806,
 0.7419354838709677,
 0.6129032258064516,
 0.5806451612903226,
 0.6451612903225806,
 0.7741935483870968,
 0.6129032258064516,
 0.4838709677419355,
 0.7741935483870968,
 0.5161290322580645,
 0.70967741

In [15]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.612903
1,0.612903
2,0.612903
3,0.709677
4,0.677419
5,0.709677
6,0.677419
7,0.419355
8,0.419355
9,0.580645


In [16]:
accuracies

[0.634453781512605,
 0.5126050420168067,
 0.6008403361344538,
 0.6218487394957983,
 0.6134453781512605,
 0.6176470588235294,
 0.634453781512605,
 0.5546218487394958,
 0.5336134453781513,
 0.592436974789916,
 0.5882352941176471,
 0.5882352941176471,
 0.6134453781512605,
 0.6218487394957983,
 0.5336134453781513,
 0.5126050420168067,
 0.6134453781512605,
 0.5672268907563025,
 0.6386554621848739,
 0.5798319327731093,
 0.5672268907563025,
 0.5756302521008403,
 0.6092436974789915,
 0.592436974789916,
 0.5798319327731093,
 0.6302521008403361,
 0.6050420168067226,
 0.6470588235294118,
 0.5630252100840336,
 0.5672268907563025,
 0.5882352941176471,
 0.6008403361344538,
 0.5840336134453782,
 0.6512605042016807,
 0.5840336134453782,
 0.5546218487394958,
 0.6218487394957983,
 0.5672268907563025,
 0.6260504201680672,
 0.5714285714285714,
 0.5966386554621849,
 0.5588235294117647,
 0.5882352941176471,
 0.5630252100840336,
 0.5672268907563025,
 0.5504201680672269,
 0.6050420168067226,
 0.52521008403361

In [17]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.634454
1,0.512605
2,0.600840
3,0.621849
4,0.613445
5,0.617647
6,0.634454
7,0.554622
8,0.533613
9,0.592437


In [18]:
precisions

[0.20212765957446807,
 0.15447154471544716,
 0.18627450980392157,
 0.21359223300970873,
 0.20388349514563106,
 0.21153846153846154,
 0.21428571428571427,
 0.12871287128712872,
 0.12264150943396226,
 0.17647058823529413,
 0.17475728155339806,
 0.18095238095238095,
 0.18556701030927836,
 0.1827956989247312,
 0.1774193548387097,
 0.13675213675213677,
 0.21495327102803738,
 0.16037735849056603,
 0.20430107526881722,
 0.19469026548672566,
 0.18421052631578946,
 0.19298245614035087,
 0.19,
 0.18269230769230768,
 0.15841584158415842,
 0.1724137931034483,
 0.18811881188118812,
 0.2376237623762376,
 0.1826086956521739,
 0.15384615384615385,
 0.18095238095238095,
 0.1595744680851064,
 0.16666666666666666,
 0.24,
 0.19642857142857142,
 0.17391304347826086,
 0.20202020202020202,
 0.17857142857142858,
 0.22115384615384615,
 0.1743119266055046,
 0.1782178217821782,
 0.17543859649122806,
 0.20869565217391303,
 0.17117117117117117,
 0.14705882352941177,
 0.1935483870967742,
 0.16842105263157894,
 0.17

In [19]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.202128
1,0.154472
2,0.186275
3,0.213592
4,0.203883
5,0.211538
6,0.214286
7,0.128713
8,0.122642
9,0.176471


In [20]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.634454,0.202128
1,0.512605,0.154472
2,0.600840,0.186275
3,0.621849,0.213592
4,0.613445,0.203883
5,0.617647,0.211538
6,0.634454,0.214286
7,0.554622,0.128713
8,0.533613,0.122642
9,0.592437,0.176471


In [21]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.634454,0.202128,0.612903
1,0.512605,0.154472,0.612903
2,0.600840,0.186275,0.612903
3,0.621849,0.213592,0.709677
4,0.613445,0.203883,0.677419
5,0.617647,0.211538,0.709677
6,0.634454,0.214286,0.677419
7,0.554622,0.128713,0.419355
8,0.533613,0.122642,0.419355
9,0.592437,0.176471,0.580645


In [22]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.638225806451612
Recall mode: 0    0.645161
dtype: float64
Recall median: 0.6451612903225806
Recall max: 0.8709677419354839
Recall min: 0.2903225806451613
Recall standard deviation: 0.08937551555021243


In [23]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.5855840336134467
Accuracy mode: 0    0.588235
dtype: float64
Accuracy median: 0.5840336134453782
Accuracy max: 0.7058823529411765
Accuracy min: 0.47478991596638653
Accuracy standard deviation: 0.033860380463549736


In [24]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.1847660907679576
Precision mode: 0    0.2
dtype: float64
Precision median: 0.18421052631578946
Precision max: 0.2696629213483146
Precision min: 0.10588235294117647
Precision standard deviation: 0.022149254571855424


In [25]:
results_df.isnull().sum().sum()

0

In [26]:
len(results_df)

1000

In [27]:
filepath = ('ML_result_files/SVC_Boomer_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)